# setup the model and load in the dataset


This part sets up the dataset

In [1]:
STUDENTID = 567     # this will be used for random states

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import logging
from utils import *
import torch.optim as optim
import os

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the data
])

# Download and load CIFAR-100 datasets
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, transform=transform, download=True)


# Calculate the sizes for train, validation, and test sets
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

# Split the train dataset into train and validation sets using a random seed
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(STUDENTID))

# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))
print("Number of test examples:", len(test_dataset))



Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000


Setting up the train, evaluation, and test functions


Setting up the neural network model


In [3]:
# creating a model that automatically runs the forward function i guess, since it is easier
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from customModels import *

In [7]:
model = NeuralNetwork()
modelName = '1FirstModelTest'

argDict = {
  'lr': 0.001,
  'maxEpoch': 250,
  'idleEpoch': 25,
  'outputName': modelName,
  'optimizer': optim.SGD(model.parameters(), lr=0.001),
  'criterion': nn.CrossEntropyLoss()
}

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, valid_loader, test_loader)
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

del model
del argDict

define the self
currently at epoch 0 train accuracy: tensor(0.0253, device='cuda:0') loss of: 4.553142375946045 eval accuracy: tensor(0.0375, device='cuda:0')
currently at epoch 1 train accuracy: tensor(0.0515, device='cuda:0') loss of: 4.296969683837891 eval accuracy: tensor(0.0617, device='cuda:0')
currently at epoch 2 train accuracy: tensor(0.0758, device='cuda:0') loss of: 4.087499937438965 eval accuracy: tensor(0.0844, device='cuda:0')
currently at epoch 3 train accuracy: tensor(0.0981, device='cuda:0') loss of: 3.9222949230194093 eval accuracy: tensor(0.1048, device='cuda:0')
currently at epoch 4 train accuracy: tensor(0.1180, device='cuda:0') loss of: 3.7862459957122803 eval accuracy: tensor(0.1206, device='cuda:0')
currently at epoch 5 train accuracy: tensor(0.1358, device='cuda:0') loss of: 3.669298194503784 eval accuracy: tensor(0.1334, device='cuda:0')
currently at epoch 6 train accuracy: tensor(0.1515, device='cuda:0') loss of: 3.57353306388855 eval accuracy: tensor(0.1455,

KeyboardInterrupt: 

In [ ]:
model = NeuralNetwork()
modelName = '2lowerLR'

argDict = {
  'lr': 0.0005,
  'maxEpoch': 250,
  'idleEpoch': 25,
  'outputName': modelName,
  'optimizer': optim.SGD(model.parameters(), lr=0.001),
  'criterion': nn.CrossEntropyLoss()
}

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, valid_loader, test_loader)
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

del model
del argDict

In [ ]:
model = NeuralNetwork()
modelName = '3rdHigherLR'

argDict = {
  'lr': 0.003,
  'maxEpoch': 250,
  'idleEpoch': 25,
  'outputName': modelName,
  'optimizer': optim.SGD(model.parameters(), lr=0.001),
  'criterion': nn.CrossEntropyLoss()
}

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, valid_loader, test_loader)
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

del model
del argDict

In [ ]:
with open('finished.txt', "w") as file:
  file.write('donezo')